In [1]:
import pandas as pd
import sys
import os
from shutil import copy2
import mwclient

try:
    from labjournal.core.databaseModel import *
except ImportError:
    sys.path.append("../../..")
    from labjournal.core.databaseModel import *

In [2]:
db_path = 'create_database/micha_add_parents.db'
db_path_out = 'create_database/micha_add_paths.db'

In [3]:
copy2(db_path,db_path_out)

In [4]:
engine = create_engine('sqlite:///{}'.format(db_path_out))
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
df_main = pd.read_sql_table(table_name=Main.__tablename__,
                            con=engine)

In [6]:
# sudo docker restart test_mediawiki_db_1
# sudo docker restart test_mediawiki_phpmyadmin_1
# sudo docker restart test_mediawiki_main_1



mediawiki_address = '134.34.112.156:8001'  # address of the server of the MediaWiki
mediawiki_protocol = 'http'      # protocol (http or https)
mediawiki_path = '/mediawiki/'   # path to MediaWiki after address

bot_username = 'Micha@service_bot'      # username
bot_password = 'o854d1i43i4dgg8teh11pdvieq725vu1'  # password

#import getpass
#bot_username = raw_input('User: ')
#bot_password = getpass.getpass("Password: ")


# Initialize the Site object
site = mwclient.Site((mediawiki_protocol, mediawiki_address), mediawiki_path)  # connect
site.login(bot_username,bot_password) # optional, if login is required

In [7]:
def get_folder(simid):
    # get the page
    page = site.pages[simid]

    # resolve redirect if possible
    page = page.resolve_redirect()

    # get the text of the page
    text = page.text()
    
    #=====================================#
    # {{folder|XXX}}
    #=====================================#
    folders=[]
    for line in text.splitlines():
        if '{{folder|' in line:
            folders.append(line.split("{{folder|",1)[1].split("}}")[0])
        elif 'folder:' in line:
            try:
                folders.append(line.split("folder:",1)[1].split("<tt>")[1].split('</tt>')[0])
            except:
                pass
    
    # fix path
    for i,folder in enumerate(folders[:]):
        if folder.startswith("SIM-PhD-King"):
            folders[i]='/home/micha/'+folder

    # get mainfolder
    main_folder=folders[0] if len(folders) > 0 else ""
    
     
    return main_folder

In [8]:
df_sub = df_main[df_main.sim_type ==  'MISSING ENTRY']
for entry_id, entry_mediawiki in zip(df_sub.index,df_sub.mediawiki):
    new_path = get_folder(entry_mediawiki)
    #print df_main.iloc[entry_id]
    #print entry_mediawiki, df_main.get_value(entry_id,'mediawiki')
    
    if os.path.exists(new_path):
        df_sub.set_value(entry_id,'path',new_path)
        df_sub.set_value(entry_id,'sim_type', 'MISSING INFO')
        #print 'updated:',entry_mediawiki, new_path
    else:
        df_sub.set_value(entry_id,'path',new_path)
        df_sub.set_value(entry_id,'sim_type', 'ERROR PATH')
        print entry_id, entry_mediawiki

377 MK0011
378 MK0012
387 MK0021
388 MK0022
423 MK0061
440 MK0086
448 MK0107
449 MK0108
450 MK0113
451 MK0114
452 MK0115
453 MK0120
454 MK0121
465 MK0188


In [9]:
session.bulk_update_mappings(
  Main,
  df_sub.to_dict(orient='records')
)

In [10]:
session.commit()
session.close()

In [11]:
#df_main